In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install nltk
!pip install --upgrade spacy torch
!python -m spacy download en_core_web_sm

## Cargar datos

#### Datos de entrenamiento

In [ ]:
import pandas as pd
#Acá agregas el archivo que contiene todos los datos para entrenar el modelo
train = pd.read_csv('datos_entrenamiento.csv', encoding='ISO-8859-1', delimiter=',') #Revisa si el caracter separador es una coma (,), un punto y coma (;) u otro
train = test[["title", "classification"]].dropna() #Train debe ser una tabla con dos columnas: "title" y "classification"
train.head()

#### Datos de prueba

In [ ]:
#Acá agregas el archivo que contiene todos aquellos datos de los cuales quieres saber su categoría
#test = pd.read_csv('datos_test.csv', encoding='ISO-8859-1', delimiter=',') #Revisa si el caracter separador es una coma (,), un punto y coma (;) u otro
test = pd.read_csv('listArticulos.csv', encoding='ISO-8859-1', delimiter=';') #Acá agregas el archivo que contiene todos aquellos datos de los cuales quieres saber su categoría
test = test[["title", "classification"]].dropna() #Test debe ser una tabla con dos columnas: "title" y "classification"
test.head()

## Procesamiento de texto

In [ ]:
# Traducir al inglés datos del set de prueba 
from googletrans import Translator
def traducir_texto(texto, idioma):
    translator = Translator()
    translated = translator.translate(texto, dest=idioma)
    return translated.text

#for i in test.index: # si se quiere traducir también los datos de entrenamiento, cambie test por datos = pd.concat([test, train], axis=1)
#  dato = test.loc[i, "title"]
#  traducido = traducir_texto(dato, "en")
#  test.loc[i, "title"] = traducido

datos = pd.concat([test, train])
datos = datos.reset_index(drop=True)

#Elminar caracteres especiales, puntuación y mayúsculas

import re
def caracteres_especiales(texto):
    muestra = r'[^a-zA-Z0-9\s]'  # Incluye todo excepto letras, números y espacios en blanco
    obslimpia = re.sub(muestra, ' ', texto)
    obslimpia = re.sub(r'\s+', ' ', obslimpia)  
    return obslimpia

for i in datos.index:
  texto = datos.loc[i, "title"]
  texto = caracteres_especiales(texto) # Quitar caracteres especiales
  texto = texto.lower() # Estandarizar a minúsculas
  datos.loc[i, "title"] = texto
    

# Eliminar stop words y lematizar palabras

import spacy
nlp = spacy.load("en_core_web_sm") #Modelo de lenguaje en inglés de spaCy
for i in datos.index:
  doc = nlp(datos.loc[i, "title"])
  palabras = [token.lemma_ for token in doc if not token.is_stop] 
  texto = ' '.join(palabras)
  datos.loc[i, "title"] = texto


# Stemming

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stemming(texto):
    obs = [stemmer.stem(token.lemma_) for token in nlp(texto)]
    return " ".join(obs)

for i in datos.index:
  datos.loc[i, "title"] = stemming(datos.loc[i, "title"])

#Limpieza extra
for i in datos.index:
  datos.loc[i, "title"] = re.sub(r'\s+', ' ', datos.loc[i, "title"])

## Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

titulo = datos["title"]
vectorizer = CountVectorizer(min_df=2) #BoW con aquellas palabras que aparecen mínimo 2 veces en el total de observacions
matriz_bow = vectorizer.fit_transform(titulo)
bow = pd.DataFrame(matriz_bow.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Dividir datos nuevamente en entrenamiento y prueba
X_test = bow.iloc[:len(test)]
X_train = bow.iloc[len(test):]
y_test = datos.iloc[:len(test), 1]
y_train = datos.iloc[len(test):, 1]

## Modelo

#### Si queremos predecir con hasta dos categorías como respuesta

In [ ]:
# Regresión logística one-vs-rest

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
clases = ['agricultural systems', 'energy systems', 'financial engineering systems', 'health systems', 'production systems', 'sustainable systems', 'transportation systems', 'urban systems']

modelrl = LogisticRegression(C= 0.1, max_iter= 100, penalty= 'l2', solver= 'lbfgs', multi_class='ovr')
modelrl.fit(X_train, y_train)
y_probs = modelrl.predict_proba(X_test)

yhat_threshold = []

for indice, prob in enumerate(y_probs): #Recorro las probabilidades obtenidas para cada observación
    clases_sobre_threshold = np.where(prob > 0.8*np.max(prob))[0] #Agrego la nueva regla de decisión
    if len(clases_sobre_threshold) == 0:
        clases_sobre_threshold = [np.argmax(prob)]

    #Si hay más de 2 categorías elegidas, selecciona aquellas dos con más probabilidad
    if len(clases_sobre_threshold) > 2:
        indices_ordenados = np.argsort(prob)[::-1]  
        clases_sobre_threshold = indices_ordenados[:2]

    yhat_threshold.append(clases_sobre_threshold)
    cat_correcta = y2_test[indice]
    convers = clases.index(cat_correcta)
    
    seleccionadas = []
    for i in clases_sobre_threshold:
        seleccionadas.append(clases[i])
    
    yhat_threshold.append(seleccionadas)

print(yhat_threshold)

In [ ]:
#Descargar los resultados como un archivo csv
yhat_threshold.to_csv('resultados_test.csv', index=False)

#### Si queremos predecir con solo una categoría de respuesta

In [ ]:
# Regresión logística one-vs-rest

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

modelrl = LogisticRegression(C= 0.1, max_iter= 100, penalty= 'l2', solver= 'lbfgs', multi_class='ovr')
modelrl.fit(X_train, y_train)
y_pred = modelrl.predict(X_test)

print(y_pred)